<h2>Utilities for testing TAP services</h2>

In [1]:
from lsst.rsp import get_tap_service
from IPython.display import display, HTML

import sys

def print_test(msg, color="black", file = None):
    display(HTML(f'<span style="color: {color}">{msg.replace(" ", "&nbsp;")}</span>'))
    if file:
        file.write(msg + '\n')

def print_error(msg, object_name, exception, query = None, file = None):
    err_msg = f"Error {msg}: {object_name}"
    display(HTML(f'<span style="color: red; font-weight: bold;">{err_msg}</span>'))
    display(HTML(f'<span style="color: red;">{exception}</span>'))
    if file:
        file.write(err_msg + '\n')
        if query:
            file.write(query + '\n')
        file.write(str(exception) + '\n')
        file.write('\n')

def test_tap_services(tap_services = ["tap", "ssotap", "consdbtap"], check_single_columns = False, log_file = None, only_print_errors = False):
    schema_query = 'SELECT * FROM tap_schema.schemas'
    for tap_service_name in tap_services:
        if not only_print_errors:
            print_test(f"Testing TAP service: {tap_service_name}", file = log_file)
        try:
            service = get_tap_service(tap_service_name)
        except Exception as e1:
            print_error("getting TAP service", tap_service_name, e1, file = log_file)
            continue
        try:
            schema_results_table = service.search(schema_query).to_table()
        except Exception as e2:
            print_error("fetching schemas for", tap_service_name, e2, file = log_file)
            continue
        for schema_name in schema_results_table['schema_name']:
            table_query = f"SELECT * FROM tap_schema.tables WHERE tap_schema.tables.schema_name = '{schema_name}' ORDER BY table_index ASC"
            try:
                table_results_table = service.search(table_query).to_table()
            except Exception as e3:
                print_error("getting tables for", schema_name, e3, file = log_file)
            for table_name in sorted(table_results_table["table_name"]):
                if not table_name.startswith("tap_schema."):
                    if not only_print_errors:
                        print_test(f"Testing table: {table_name}", file = log_file)
                    test_query = f'SELECT * FROM {table_name} LIMIT 1'
                    if not only_print_errors:
                        print_test(test_query, file = log_file)
                    try:
                        test_results_table = service.search(test_query).to_table()
                        if not only_print_errors:
                            print_test(f"Successfully got {len(test_results_table)} record(s) from {table_name}", color="green", file = log_file) 
                    except Exception as e4:
                        print_error("selecting records from", table_name, e4, file = log_file)
                    if check_single_columns:
                        columns_query = f"SELECT * FROM tap_schema.columns WHERE table_name = '{table_name}'"
                        if not only_print_errors:
                            print_test(columns_query)
                        columns_result = service.search(columns_query).to_table()
                        sorted_column_list = sorted(columns_result["column_name"])
                        if not only_print_errors:
                            print_test("Column list:", file = log_file)
                        for column_name in sorted_column_list:
                            if not only_print_errors:
                                print_test(f"    {column_name}", file = log_file)  
                        for column_name in sorted_column_list:
                            if not only_print_errors:
                                print_test(f"Checking single column: {table_name}.{column_name}", file = log_file)
                            single_column_query = f'SELECT "{column_name}" from {table_name} LIMIT 1'
                            if not only_print_errors:
                                print_test(single_column_query, file = log_file)
                            try:
                                single_column_results_table = service.search(single_column_query).to_table()
                                if not only_print_errors:
                                    print_test(f"Successfully got results for single column: {table_name}.{column_name}", color="green", file = log_file)
                            except Exception as e5:
                                print_error("getting results for column", f"{table_name}.{column_name}", e5, query = single_column_query, file = log_file)
                                continue
        print_test("Done checking TAP services!")

<h2>All TAP services</h2>

In [2]:
# , "livetap",
test_tap_services(["tap", "consdbtap"])

<h2>consdbtap</h2>

In [ ]:
with open("consdbtap_test.txt", 'w') as f:
    test_tap_services(["consdbtap"], check_single_columns=True, log_file = f)

<h2>Debugging junk below here</h2>

In [ ]:
# test_tap_services()

In [ ]:
# Query for browser:
# https://usdf-rsp-int.slac.stanford.edu/api/consdbtap/sync?LANG=ADQL&REQUEST=doQuery&QUERY=SELECT+*+FROM+cdb_latiss.visit1+LIMIT+1

In [ ]:
# service = get_tap_service("consdbtap")
# results = service.search("SELECT * FROM cdb_latiss.visit1_quicklook").to_table()
# results

In [113]:
service = get_tap_service("consdbtap")
type(service)

pyvo.dal.tap.TAPService

In [107]:
service.search("SELECT s_region from cdb_lsstcomcamsim.ccdvisit1")

<DALResultsTable length=58221>
s_region
str1024 
--------
        
        
        
        
        
        
        
        
        
     ...
        
        
        
        
        
        
        
        
        
        

In [ ]:
# test of transaction issue
service = get_tap_service("consdbtap")

schema_results_table = service.search('SELECT * FROM tap_schema.schemas').to_table()
for schema_name in schema_results_table['schema_name']:
    table_query = f"SELECT * FROM tap_schema.tables WHERE tap_schema.tables.schema_name = '{schema_name}' ORDER BY table_name ASC"
    table_results = service.search(table_query).to_table()
    for table_name in table_results["table_name"]:
        if not table_name.startswith("tap_schema."):
            select_all_query = f'SELECT * FROM {table_name} LIMIT 1'
            try:
                service.search(select_all_query)
            except Exception as e:
                print(select_all_query)
                print(e)
                print('\n')
        column_query = f"SELECT * FROM tap_schema.columns WHERE table_name = '{table_name}' ORDER BY table_name ASC"
        column_results = service.search(column_query).to_table()
        for column_name in column_results["column_name"]:
            select_column_query = f'SELECT "{column_name}" from {table_name} LIMIT 1'
            try:
                service.search(select_column_query)
            except Exception as e:
                print(select_column_query)
                print(e)
                print('\n')

In [129]:
schema_results_table = get_tap_service("consdbtap").search('SELECT * FROM tap_schema.schemas').to_table()
for schema_name in schema_results_table['schema_name']:
    table_query = f"SELECT * FROM tap_schema.tables WHERE tap_schema.tables.schema_name = '{schema_name}' ORDER BY table_name ASC"
    table_results = get_tap_service("consdbtap").search(table_query).to_table()
    for table_name in table_results["table_name"]:
        if not table_name.startswith("tap_schema."):
            column_query = f"SELECT * FROM tap_schema.columns WHERE table_name = '{table_name}' ORDER BY table_name ASC"
            column_results = service.search(column_query).to_table()
            for column_name in column_results["column_name"]:
                select_column_query = f'SELECT "{column_name}" from {table_name} LIMIT 1'
                try:
                    get_tap_service("consdbtap").search(select_column_query)
                except Exception as e:
                    print(select_column_query)
                    print(e)


SELECT "ccdexposure_id" from cdb_lsstcomcamsim.ccdexposure LIMIT 1
ERROR: permission denied for table ccdexposure
SELECT "day_obs" from cdb_lsstcomcamsim.ccdexposure LIMIT 1
ERROR: current transaction is aborted, commands ignored until end of transaction block ERROR: permission denied for table ccdexposure
SELECT "detector" from cdb_lsstcomcamsim.ccdexposure LIMIT 1
ERROR: current transaction is aborted, commands ignored until end of transaction block ERROR: permission denied for table ccdexposure
SELECT "exposure_id" from cdb_lsstcomcamsim.ccdexposure LIMIT 1
ERROR: current transaction is aborted, commands ignored until end of transaction block ERROR: permission denied for table ccdexposure
SELECT "s_region" from cdb_lsstcomcamsim.ccdexposure LIMIT 1
ERROR: current transaction is aborted, commands ignored until end of transaction block ERROR: permission denied for table ccdexposure
SELECT "seq_num" from cdb_lsstcomcamsim.ccdexposure LIMIT 1
ERROR: current transaction is aborted, comma

In [130]:
schema_results_table = service.search('SELECT "value" from cdb_lsstcomcamsim.ccdexposure_flexdata LIMIT 1').to_table()

DALQueryError: ERROR: permission denied for table ccdexposure_flexdata

In [125]:
schema_results_table = get_tap_service("consdbtap").search('SELECT "value" from cdb_lsstcomcamsim.ccdexposure_flexdata LIMIT 1').to_table()

DALQueryError: ERROR: permission denied for table ccdexposure_flexdata

In [127]:
get_tap_service("consdbtap").search('SELECT "value" from cdb_lsstcomcamsim.ccdexposure LIMIT 1').to_table()

DALQueryError: validateColumnNonAlias: Column: ["value"] does not exist.

In [131]:
get_tap_service("consdbtap").search('SELECT "day_obs" from cdb_lsstcomcamsim.ccdexposure LIMIT 1')

DALQueryError: ERROR: permission denied for table ccdexposure